# {} Vehicle Position Data Available for Scheduled Data

In [1]:
import utils
import shared_utils
from dla_utils import _dla_utils as dla_utils

from siuba import *
import pandas as pd

from IPython.display import display, Markdown

import altair as alt

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#agency is defined as the single operator (a unique calitp_itp_id) used to subset the data

In [3]:
#parameter cell
calitp_id = 300

In [4]:
rt_sched = utils.read_data()


In [5]:
rt_sched['calitp_deleted_at'] = pd.to_datetime(rt_sched['calitp_deleted_at'])


In [20]:
## for now filtering down these columns so we do not have the bad 
## moving to utils in the read_data()

# rt_sched = rt_sched[((rt_sched["calitp_itp_id"]==290) & (rt_sched["calitp_url_number"]==1)) | ((rt_sched["calitp_itp_id"]==300))]

In [7]:
#getting df with daily pct 
rt_sched_date = utils.agg_by_date(rt_sched, 'num_sched', 'num_vp')

In [8]:
agency = rt_sched>>filter(_.calitp_itp_id == calitp_id)

In [9]:
#getting df with daily pct for single agency
agency_date = utils.agg_by_date(agency, 'num_sched', 'num_vp')

In [10]:
agency_date.head()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,300,Big Blue Bus,0,2022-05-01,Sunday,May,770,526,0.68
1,300,Big Blue Bus,0,2022-05-02,Monday,May,1516,1066,0.70
2,300,Big Blue Bus,0,2022-05-03,Tuesday,May,1516,1365,0.90
3,300,Big Blue Bus,0,2022-05-04,Wednesday,May,1516,1381,0.91
4,300,Big Blue Bus,0,2022-05-05,Thursday,May,1516,1343,0.89


In [14]:
utils.get_agg_pct(rt_sched,
                groupings = ['calitp_itp_id'],
                sum_sched = 'num_sched',
                sum_vp = 'num_vp')

,calitp_itp_id,num_sched,num_vp,avg
0,290,75860,55245,0.73
1,300,78765,65968,0.84


In [ ]:
# average vp for sched trips for all operators over all days
((rt_sched >>summarize(tot_sched =_.num_sched.sum(),
                       tot_vp = _.num_vp.sum())) >> mutate(avg = _.tot_vp/_.tot_sched))

,tot_sched,tot_vp,avg
0,154625,121213,0.78


In [18]:
utils.total_average_with_1op_chart(rt_sched, 300)

alt.Chart(...)

In [19]:
bar = (alt.Chart(agency_date)
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date:T', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp:Q', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                #scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title= "Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(agency_date
                ).mark_rule(color='red'
                           ).encode(y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%')))

(bar + rule).properties(width=800)

alt.LayerChart(...)